# Xarray-spatial
### User Guide: Mahalanobis Distance
-----

**Mahalanobis distance** measures how unusual a pixel is compared to a
reference population, *accounting for correlations between bands*.

The formula is:

$$D(\mathbf{x}) = \sqrt{(\mathbf{x} - \boldsymbol{\mu})^T \, \Sigma^{-1} \, (\mathbf{x} - \boldsymbol{\mu})}$$

where **x** is the pixel vector, **μ** is the mean of the reference
distribution, and **Σ⁻¹** is the inverse covariance matrix.

That’s the only time we’ll show the formula — the rest of this notebook
is visual.

-----

[The problem with Euclidean distance](#The-Problem-with-Euclidean-Distance)

[Mahalanobis fixes this](#Mahalanobis-Fixes-This)

[From scatter plot to raster](#From-Scatter-Plot-to-Raster)

[Apply Mahalanobis distance](#Apply-Mahalanobis-Distance)

[Train-apply workflow](#Train-Apply-Workflow)

[Accessor syntax](#Accessor-Syntax)

[Summary](#Summary)

-----

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from xrspatial import mahalanobis
import xrspatial  # registers .xrs accessor

np.random.seed(42)

## The Problem with Euclidean Distance

Imagine you have two spectral bands (NIR and Red) for a patch of healthy
vegetation. The reflectance values are correlated — when NIR goes up, Red
tends to go up a bit too.

If we draw **Euclidean iso-distance circles** around the mean, a point
that falls *along* the correlation axis gets the same score as a point
that falls *perpendicular* to it. But one is a normal vegetation pixel
and the other is anomalous. Euclidean distance can’t tell them apart.

In [ ]:
# Generate correlated 2D data (healthy vegetation: NIR vs Red)
n_points = 500
mean_2d = np.array([0.45, 0.15])  # NIR high, Red low
cov_2d = np.array([[0.010, 0.006],
                    [0.006, 0.005]])
data_2d = np.random.multivariate_normal(mean_2d, cov_2d, n_points)

# Two test points at the same Euclidean distance from the mean
# but very different in terms of the population
eigvals, eigvecs = np.linalg.eigh(cov_2d)
r = 0.16
point_along = mean_2d + r * eigvecs[:, 1]       # along major axis (normal)
point_perp = mean_2d + r * eigvecs[:, 0]         # along minor axis (unusual)

fig, ax = plt.subplots(figsize=(7, 6))
ax.scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.3, s=10, c='steelblue',
           label='Healthy vegetation')

# Euclidean iso-distance circles
for radius in [0.08, 0.12, 0.16]:
    circle = plt.Circle(mean_2d, radius, fill=False, color='grey',
                        linestyle='--', linewidth=1)
    ax.add_patch(circle)

ax.plot(*point_along, 'o', ms=12, color='green', zorder=5,
        label=f'Point A (along correlation) — Euclid = {r:.2f}')
ax.plot(*point_perp, 's', ms=12, color='red', zorder=5,
        label=f'Point B (perpendicular)      — Euclid = {r:.2f}')

ax.set_xlabel('NIR reflectance')
ax.set_ylabel('Red reflectance')
ax.set_title('Euclidean distance treats both points the same')
ax.legend(loc='upper left', fontsize=9)
ax.set_aspect('equal')
ax.set_xlim(0.2, 0.7)
ax.set_ylim(-0.05, 0.35)
plt.tight_layout()
plt.show()

Both points sit on the same grey circle — Euclidean distance says they’re
equally far from the mean. But Point B (red square) is clearly *outside*
the cloud of normal data.

## Mahalanobis Fixes This

Mahalanobis distance uses the **covariance** of the data to stretch the
distance metric. Instead of circles, we get **ellipses** that follow the
shape of the data cloud. Now the two points get different scores.

In [ ]:
# Compute Mahalanobis distance for every point in the scatter
inv_cov_2d = np.linalg.inv(cov_2d)

def mahal_2d(pts, mu, inv_c):
    diff = pts - mu
    return np.sqrt(np.sum(diff @ inv_c * diff, axis=1))

md_data = mahal_2d(data_2d, mean_2d, inv_cov_2d)
md_A = mahal_2d(point_along.reshape(1, -1), mean_2d, inv_cov_2d)[0]
md_B = mahal_2d(point_perp.reshape(1, -1), mean_2d, inv_cov_2d)[0]

fig, ax = plt.subplots(figsize=(7, 6))
sc = ax.scatter(data_2d[:, 0], data_2d[:, 1], c=md_data, cmap='viridis',
                alpha=0.5, s=10)
plt.colorbar(sc, ax=ax, label='Mahalanobis distance')

# Mahalanobis iso-distance ellipses
for n_std in [1, 2, 3]:
    w, h = 2 * n_std * np.sqrt(eigvals)
    angle = np.degrees(np.arctan2(eigvecs[1, 1], eigvecs[0, 1]))
    ell = Ellipse(mean_2d, w, h, angle=angle, fill=False,
                  color='grey', linestyle='--', linewidth=1)
    ax.add_patch(ell)

ax.plot(*point_along, 'o', ms=12, color='green', zorder=5,
        label=f'Point A (along correlation) — Mahal = {md_A:.1f}')
ax.plot(*point_perp, 's', ms=12, color='red', zorder=5,
        label=f'Point B (perpendicular)      — Mahal = {md_B:.1f}')

ax.set_xlabel('NIR reflectance')
ax.set_ylabel('Red reflectance')
ax.set_title('Mahalanobis distance separates normal from anomalous')
ax.legend(loc='upper left', fontsize=9)
ax.set_aspect('equal')
ax.set_xlim(0.2, 0.7)
ax.set_ylim(-0.05, 0.35)
plt.tight_layout()
plt.show()

**This is the key insight.** Point A sits comfortably inside the 2σ
ellipse (low Mahalanobis distance) while Point B is pushed far outside
(high Mahalanobis distance). Same Euclidean distance, very different
Mahalanobis distances.

## From Scatter Plot to Raster

Now let’s move from 2D scatter plots to raster data. We’ll create a
synthetic 200×200 landscape with 3 correlated bands (simulating NIR,
Red, Green reflectance). Then we’ll insert an anomaly patch where the
band relationships are different.

In [ ]:
np.random.seed(42)

H, W = 200, 200

# Background: correlated bands (healthy vegetation)
mu_bg = np.array([0.45, 0.15, 0.10])  # NIR, Red, Green
cov_bg = np.array([[0.010, 0.004, 0.003],
                    [0.004, 0.005, 0.002],
                    [0.003, 0.002, 0.004]])

pixels_bg = np.random.multivariate_normal(mu_bg, cov_bg, H * W)
pixels_bg = np.clip(pixels_bg, 0, 1)

# Reshape to (3, H, W)
bands_3d = pixels_bg.T.reshape(3, H, W)

# Anomaly patch (60x40 rectangle): different correlation structure
# (e.g. bare soil — lower NIR, higher Red)
mu_anom = np.array([0.25, 0.28, 0.22])
cov_anom = np.array([[0.004, -0.002, 0.001],
                      [-0.002, 0.006, 0.001],
                      [0.001, 0.001, 0.003]])
patch_h, patch_w = 60, 40
r0, c0 = 100, 120
n_patch = patch_h * patch_w
pixels_anom = np.random.multivariate_normal(mu_anom, cov_anom, n_patch)
pixels_anom = np.clip(pixels_anom, 0, 1)
bands_3d[:, r0:r0+patch_h, c0:c0+patch_w] = pixels_anom.T.reshape(3, patch_h, patch_w)

# Create xarray DataArrays (one per band)
band_names = ['NIR', 'Red', 'Green']
bands = []
for i, name in enumerate(band_names):
    da = xr.DataArray(bands_3d[i], dims=['y', 'x'], name=name)
    bands.append(da)

# Visualize the 3 bands
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for i, (b, name) in enumerate(zip(bands, band_names)):
    im = axes[i].imshow(b.values, cmap='gray', vmin=0, vmax=0.6)
    axes[i].set_title(f'Band {i+1}: {name}')
    axes[i].set_axis_off()
    plt.colorbar(im, ax=axes[i], fraction=0.046)
plt.suptitle('Synthetic 3-band raster with embedded anomaly', fontsize=13)
plt.tight_layout()
plt.show()

The anomaly patch (lower-right area) is hard to spot in individual bands
— its values overlap with the background range. The difference is in the
*correlation structure* between bands, not in individual band values.

## Apply Mahalanobis Distance

Now let’s use `xrspatial.mahalanobis` to compute the per-pixel distance.
When called without explicit `mean` / `inv_cov`, it computes statistics
from all valid pixels in the input bands.

In [ ]:
result = mahalanobis(bands)

fig, ax = plt.subplots(figsize=(7, 5))
im = ax.imshow(result.values, cmap='magma')
plt.colorbar(im, ax=ax, label='Mahalanobis distance')
ax.set_title('Mahalanobis distance (stats from all pixels)')
ax.set_axis_off()
plt.tight_layout()
plt.show()

The anomaly patch lights up clearly. Even though no single band showed
an obvious difference, the *multivariate* distance picks it up because
the *relationship* between bands is different there.

Let’s compare with per-band z-scores to drive this point home.

In [ ]:
# Per-band z-scores
fig, axes = plt.subplots(1, 4, figsize=(18, 4))
for i, (b, name) in enumerate(zip(bands, band_names)):
    z = (b - float(b.mean())) / float(b.std())
    im = axes[i].imshow(z.values, cmap='RdBu_r', vmin=-3, vmax=3)
    axes[i].set_title(f'{name} z-score')
    axes[i].set_axis_off()
    plt.colorbar(im, ax=axes[i], fraction=0.046)

im = axes[3].imshow(result.values, cmap='magma')
axes[3].set_title('Mahalanobis distance')
axes[3].set_axis_off()
plt.colorbar(im, ax=axes[3], fraction=0.046)

plt.suptitle('Per-band z-scores vs. Mahalanobis distance', fontsize=13)
plt.tight_layout()
plt.show()

The z-score maps show the anomaly patch blending with the background
— no single band cleanly isolates it. Mahalanobis distance (right panel)
detects it because it considers *all bands jointly*.

## Train-Apply Workflow

In practice you often want to compute statistics from a **training region**
(e.g. a known land-cover type) and then score the entire image against
that reference. Pixels similar to the training region will score low;
dissimilar pixels will score high.

We’ll use the top-left quadrant (all background, no anomaly) as our
training region.

In [ ]:
# Extract training pixels from the top-left quadrant
train_region = np.stack([b.values[:100, :100] for b in bands])  # (3, 100, 100)
train_flat = train_region.reshape(3, -1).T  # (10000, 3)

# Compute reference statistics
mu = train_flat.mean(axis=0)
cov = np.cov(train_flat, rowvar=False)
inv_cov = np.linalg.inv(cov)

print('Training mean:', np.round(mu, 4))
print('Training covariance:\n', np.round(cov, 5))

In [ ]:
# Apply training stats to the full raster
result_trained = mahalanobis(bands, mean=mu, inv_cov=inv_cov)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

im0 = axes[0].imshow(result.values, cmap='magma')
axes[0].set_title('Auto stats (all pixels)')
axes[0].set_axis_off()
plt.colorbar(im0, ax=axes[0], label='Mahalanobis distance')

im1 = axes[1].imshow(result_trained.values, cmap='magma')
axes[1].set_title('Training stats (top-left quadrant)')
axes[1].set_axis_off()
plt.colorbar(im1, ax=axes[1], label='Mahalanobis distance')

# Outline the training region
from matplotlib.patches import Rectangle
rect = Rectangle((0, 0), 100, 100, linewidth=2, edgecolor='cyan',
                  facecolor='none', linestyle='--')
axes[1].add_patch(rect)
axes[1].text(5, 10, 'training\nregion', color='cyan', fontsize=9, va='top')

plt.suptitle('Auto stats vs. train-apply workflow', fontsize=13)
plt.tight_layout()
plt.show()

The train-apply result (right) uses statistics from only the training
region (dashed cyan box), so the anomaly patch stands out even more
sharply. This workflow is useful when you have a known reference class
and want to find pixels that deviate from it.

## Accessor Syntax

As an alternative to the function call, you can use the `.xrs` accessor.
The first band is the calling DataArray; the remaining bands are passed
as a list.

In [ ]:
# Equivalent to: mahalanobis([bands[0], bands[1], bands[2]])
result_acc = bands[0].xrs.mahalanobis([bands[1], bands[2]])

# Verify they are identical
print('Results match:', np.allclose(result.values, result_acc.values))

The accessor also accepts `mean` and `inv_cov` keyword arguments:
```python
bands[0].xrs.mahalanobis([bands[1], bands[2]], mean=mu, inv_cov=inv_cov)
```

## Summary

**When to use Mahalanobis distance:**
- **Anomaly detection** — find pixels that don’t fit the expected spectral pattern
- **Habitat suitability** — score pixels by similarity to a reference habitat
- **Change detection** — compute distance from a pre-change baseline
- **Spectral classification** — distance to each class mean (minimum-distance classifier)

**Key parameters:**
| Parameter | Description |
|-----------|-------------|
| `bands` | List of 2-D DataArrays (at least 2) |
| `mean` | Reference mean vector (auto-computed if omitted) |
| `inv_cov` | Inverse covariance matrix (auto-computed if omitted) |
| `name` | Name for the output DataArray (default: `'mahalanobis'`) |

Both `mean` and `inv_cov` must be provided together, or both omitted.